<a href="https://colab.research.google.com/github/ykitaguchi77/statistics_for_articles/blob/main/Canalicular_excision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [88]:
import pandas as pd
import codecs
from scipy import stats

In [104]:
csv_path = "/content/drive/MyDrive/作成中の論文/涙小管切断/涙小管切除症例.csv"

#Open csv
#with codecs.open(csv_path, "r", "Shift-JIS", "ignore") as file:
with codecs.open(csv_path, "r", "UTF-8", "ignore") as file:
        df = pd.read_csv(file, index_col=None, header=0)

#**フォロー期間12ヶ月以上を抜き出し**

In [ ]:
df_long = df[df['Observation period (month)'] >= 12]
len(df_long)
#df_long.columns
df_long

In [106]:
df_long.value_counts("Side")

Side
R    14
L    12
dtype: int64

In [103]:
df_long.value_counts("History")

History
Punctal occulusion    5
dtype: int64

In [81]:
df_long.describe()["Pre Schirmer I"]

count    25.000000
mean      3.000000
std       2.327373
min       0.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       8.000000
Name: Pre Schirmer I, dtype: float64

In [82]:
df_long.value_counts("History")

History
Punctal occulusion    5
dtype: int64

In [90]:
print(df_long.describe()["Fluo_D_pre"])
print(df_long.value_counts("Fluo_D_pre"))

count    26.000000
mean      2.500000
std       0.583095
min       1.000000
25%       2.000000
50%       3.000000
75%       3.000000
max       3.000000
Name: Fluo_D_pre, dtype: float64
Fluo_D_pre
3.0    14
2.0    11
1.0     1
dtype: int64


In [91]:
print(df_long.describe()["Fluo_D_post"])
print(df_long.value_counts("Fluo_D_post"))

count    26.000000
mean      1.423077
std       0.986836
min       0.000000
25%       1.000000
50%       2.000000
75%       2.000000
max       3.000000
Name: Fluo_D_post, dtype: float64
Fluo_D_post
2.0    11
0.0     6
1.0     6
3.0     3
dtype: int64


In [93]:
print(df_long.describe()["Filamentosa_pre"])
print(df_long.value_counts("Filamentosa_pre"))

count    26.000000
mean      0.192308
std       0.401918
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000
Name: Filamentosa_pre, dtype: float64
Filamentosa_pre
0.0    21
1.0     5
dtype: int64


In [94]:
print(df_long.describe()["Filamentosa_post"])
print(df_long.value_counts("Filamentosa_post"))

count    26.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: Filamentosa_post, dtype: float64
Filamentosa_post
0.0    26
dtype: int64


In [111]:
print(df_long.describe()["Pre Schirmer I"])
print(df_long.value_counts("Pre Schirmer I"))


count    25.000000
mean      3.000000
std       2.327373
min       0.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       8.000000
Name: Pre Schirmer I, dtype: float64
Pre Schirmer I
2.0    7
1.0    6
3.0    5
7.0    4
0.0    1
4.0    1
8.0    1
dtype: int64


In [95]:
print(df_long.describe()["Recurrent period (month)"])


count    5.000000
mean     3.800000
std      2.774887
min      1.000000
25%      2.000000
50%      3.000000
75%      5.000000
max      8.000000
Name: Recurrent period (month), dtype: float64


In [89]:
stats.wilcoxon(df_long['Fluo_D_pre'], df_long['Fluo_D_post'])

WilcoxonResult(statistic=4.5, pvalue=0.00031767360182023084)

#**患者別**

In [ ]:
df_long_pts = df_long.drop_duplicates(subset=['Patient NO'], keep="last")
len(df_long_pts)
df_long_pts


In [115]:
df_long_pts.describe()["Age (yrs)"]

count    18.000000
mean     60.388889
std      15.193415
min      37.000000
25%      49.250000
50%      59.500000
75%      73.750000
max      86.000000
Name: Age (yrs), dtype: float64

In [118]:
print(df_long_pts.value_counts("Background disease"))

Background disease
Sjogren             7
GVHD                5
BSK                 1
Post RT             1
RA+ドライアイ、角膜感染       1
Sjogren, SLE, RA    1
Sjogren未            1
強皮                  1
dtype: int64


In [ ]:
"""

M/F
F    13
M     5
dtype: int64


Background disease
Sjogren             6
GVHD                5
BSK                 1
Post RT             1
RA+ドライアイ、角膜感染       1
Shogren             1
Shogren, SLE, RA    1
Shogren未検           1
強皮症                 1
dtype: int64


History
Punctal occulusion    4
dtype: int64


Side
L    12
R     6
dtype: int64


Observation period (month)
28.0    3
12.0    2
29.0    2
13.0    1
14.0    1
15.0    1
16.0    1
19.0    1
23.0    1
25.0    1
27.0    1
30.0    1
32.0    1
36.0    1
dtype: int64
"""